In [1]:
import requests

# Function to fetch and transform destination data
def fetch_and_transform_destination_data(destination_api_key, destination_api_url):
    headers = {
        "accept": "application/json",
        "x-api-key": destination_api_key
    }

    destination_list = []

    for page in range(5): #sometimes if fails due to the large number here, might try smaller (e.g. 5), or just rerun
        url = f"{destination_api_url}&page={page}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()

            for entry in data['data']:
                name = entry.get('name', None)
                geo = entry.get('geo', {})
                latitude = geo.get('latitude', None)
                longitude = geo.get('longitude', None)
                classifications = entry.get('classification', [])

                classification_list = []

                for classification in classifications:
                    classification_name = classification.get('name', None)
                    classification_values = classification.get('values', [])

                    for value in classification_values:
                        value_name = value.get('name', None)
                        value_title = value.get('title', None)

                        classification_list.append({
                            "classification_name": classification_name,
                            "value_name": value_name,
                            "value_title": value_title
                        })

                destination_dict = {
                    "name": name,
                    "latitude": latitude,
                    "longitude": longitude,
                    "classifications": classification_list
                }

                destination_list.append(destination_dict)
        else:
            print("Failed to retrieve data")

    return destination_list

# Function to fetch location data based on coordinates
def fetch_location_data(location_api_key, destination_list):
    API_GEO = "https://dev.virtualearth.net/REST/v1/Locations/"
    location_data = {}

    for destination in destination_list:
        name = destination['name']
        latitude = destination['latitude']
        longitude = destination['longitude']
        api_url = f"{API_GEO}{latitude},{longitude}?key={location_api_key}"
        response = requests.get(api_url)

        if response.status_code == 200:
            geo_data = response.json()

            if 'resourceSets' in geo_data and len(geo_data['resourceSets']) > 0:
                resource = geo_data['resourceSets'][0]['resources'][0]
                canton = resource.get('address', {}).get('adminDistrict', 'Unknown')
                municipality = resource.get('address', {}).get('adminDistrict2', 'Unknown')

                if canton not in location_data:
                    location_data[canton] = {}

                if municipality not in location_data[canton]:
                    location_data[canton][municipality] = []

                location_data[canton][municipality].append(name)
            else:
                print(f"Failed to retrieve data for {name}")
        else:
            print(f"Failed to retrieve data for {name}")

    location_list = []

    for canton, municipalities in location_data.items():
        for municipality, destinations in municipalities.items():
            location_dict = {
                "municipality": municipality,
                "canton": canton,
                "destinations": destinations
            }
            location_list.append(location_dict)

    return location_list

# Example usage
destination_api_key = 'HFjqHMLW8y8HdSOUjKBfL9JZYPQo5Rvo3Qq1yk3z'
location_api_key = 'AhxU1TpHSGQtGn7LTJyhZFdvHo-DujmZRmBarW5YqZ-ez_Rx-oWAooP6JBmizGzI'
destination_api_url = 'https://opendata.myswitzerland.io/v1/destinations/?hitsPerPage=50&striphtml=true'

destination_list = fetch_and_transform_destination_data(destination_api_key, destination_api_url)
location_list = fetch_location_data(location_api_key, destination_list)

In [3]:
print(destination_list[0]) #to test how the data looks 
print(location_list[0])

{'name': 'Lake Staz', 'latitude': 46.49853723585918, 'longitude': 9.83939900062391, 'classifications': [{'classification_name': 'licences', 'value_name': 'bysa', 'value_title': 'BY-SA'}, {'classification_name': 'placetypes', 'value_name': 'mountainlakes', 'value_title': 'Mountain Lakes'}]}
{'municipality': 'Maloja/Maloggia', 'canton': 'Graubünden', 'destinations': ['Lake Staz']}


In [4]:
import sqlite3

# Connect to the SQLite database (replace 'your_database.db' with your actual database file)
conn = sqlite3.connect('locations.db')
cursor = conn.cursor()

# Create the municipalities table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS municipalities (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        canton TEXT
    )
''')

# Create the destinations table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS destinations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        municipality_id INTEGER,
        name TEXT,
        FOREIGN KEY (municipality_id) REFERENCES municipalities (id)
    )
''')

# Iterate through location_list and insert data into tables
for location in location_list:
    # Insert into municipalities table
    cursor.execute('''
        INSERT INTO municipalities (name, canton)
        VALUES (?, ?)
    ''', (location['municipality'], location['canton']))
    
    # Get the ID of the last inserted row (municipality)
    municipality_id = cursor.lastrowid
    
    # Insert into destinations table
    destinations = location.get('destinations', [])
    for destination in destinations:
        cursor.execute('''
            INSERT INTO destinations (municipality_id, name)
            VALUES (?, ?)
        ''', (municipality_id, destination))

# Commit changes and close the connection
conn.commit()
conn.close()


In [5]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('locations.db')
cur = conn.cursor()

# Execute a SELECT query to retrieve data from the table
cur.execute("SELECT * FROM municipalities")

# Fetch all rows from the result set
rows = cur.fetchall()

# Display the rows
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()


(1, 'Maloja/Maloggia', 'Graubünden')
(2, 'Plessur', 'Graubünden')
(3, 'Prättigau-Davos', 'Graubünden')
(4, 'Viamala Region', 'Graubünden')
(5, 'Moesa', 'Graubünden')
(6, 'Albula', 'Graubünden')
(7, 'Surselva', 'Graubünden')
(8, 'Landquart', 'Graubünden')
(9, 'Werdenberg', 'St Gallen')
(10, 'Toggenburg', 'St Gallen')
(11, 'Sargans', 'St Gallen')
(12, 'See-Gaster', 'St Gallen')
(13, 'Waldenburg', 'Basel-Country')
(14, 'Laufen', 'Basel-Country')
(15, 'Sissach', 'Basel-Country')
(16, 'Schwyz', 'Schwyz')
(17, 'March', 'Schwyz')
(18, 'Weinfelden', 'Thurgau')
(19, 'Kreuzlingen', 'Thurgau')
(20, 'Frauenfeld', 'Thurgau')
(21, 'Münchwilen', 'Thurgau')
(22, 'Arbon', 'Thurgau')
(23, 'Seeland', 'Berne')
(24, 'Bernese Jura', 'Berne')
(25, 'Bern-Mittelland', 'Berne')
(26, 'Frutigen-Niedersimmental', 'Berne')
(27, 'Obersimmental-Saanen', 'Berne')
(28, 'Emmental', 'Berne')
(29, 'Oberaargau', 'Berne')
(30, 'Interlaken-Oberhasli', 'Berne')
(31, 'Thun', 'Berne')
(32, 'Biel', 'Berne')
(33, 'Raron', 'Valais

In [6]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('locations.db')
cur = conn.cursor()

# Execute a SELECT query to retrieve data from the table
cur.execute("SELECT * FROM destinations")

# Fetch all rows from the result set
rows = cur.fetchall()

# Display the rows
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

(1, 1, 'Lake Staz')
(2, 2, 'Arosa')
(3, 2, 'Molinis')
(4, 3, 'St. Antönien')
(5, 3, 'Fideriser Heuberge')
(6, 3, 'Family resort Davos Klosters')
(7, 4, 'Andeer')
(8, 4, 'Flerden')
(9, 4, 'Casti-Wergenstein')
(10, 4, 'Rodels')
(11, 4, 'Arezen')
(12, 4, 'Tartar')
(13, 4, 'Präz')
(14, 5, 'Mesocco')
(15, 5, 'Braggio')
(16, 5, 'Selna-Landarenca')
(17, 6, 'Lantsch / Lenz')
(18, 6, 'Latsch')
(19, 7, 'Surselva')
(20, 8, 'Says')
(21, 9, 'Salez')
(22, 9, 'Trübbach')
(23, 10, 'Necker')
(24, 10, 'Neu St. Johann')
(25, 10, 'Unterrindal')
(26, 10, 'Hoffeld')
(27, 11, 'Sargans')
(28, 12, 'Uznach')
(29, 12, 'Gommiswald')
(30, 12, 'Uetliburg SG')
(31, 13, 'Diegten')
(32, 14, 'Burg im Leimental')
(33, 15, 'Itingen')
(34, 15, 'Anwil')
(35, 15, 'Nusshof')
(36, 16, 'Schwyz')
(37, 16, 'Riemenstalden')
(38, 16, 'Hoch-Ybrig')
(39, 16, 'Ibergeregg')
(40, 17, 'Altendorf')
(41, 18, 'Stehrenberg')
(42, 18, 'Leimbach TG')
(43, 18, 'Mauren TG')
(44, 18, 'St. Pelagiberg TG')
(45, 18, 'Birwinken')
(46, 18, 'Hohentann